# Benthic Habitat Mapping with Hyperspectral Remote Sensing

## Mapping the Seafloor from Space Using Planet Tanager Data

---

**The Science Question:**

> *"Can we map what's on the seafloor in shallow coastal waters using hyperspectral imagery - and does having 400+ spectral bands help compared to traditional multispectral sensors?"*

---

In this notebook, we'll explore an exciting application of hyperspectral remote sensing: **seeing through water to map the seafloor**.

In clear, shallow coastal waters, sunlight penetrates to the bottom, reflects off the substrate (sand, seagrass, coral, algae), and travels back up through the water column to a satellite sensor. Different bottom types have distinct spectral signatures - unique "fingerprints" of how they reflect light at different wavelengths.

With traditional **multispectral** sensors like Landsat or Sentinel-2 (3-10 broad bands), these signatures can look frustratingly similar. But with **hyperspectral** data from Planet's Tanager satellite (426 narrow bands), we can detect subtle absorption features that distinguish between different habitats.

### Why Does This Matter?

- **Seagrass meadows** are one of the most effective carbon sinks on Earth, storing carbon 35x faster than tropical rainforests
- **Coastal habitat monitoring** is critical for fisheries management and biodiversity conservation
- **Climate change** and human activities are rapidly altering shallow marine ecosystems
- Traditional field surveys are expensive, time-consuming, and can only cover small areas

Satellite-based mapping offers a scalable solution - but only if we can accurately distinguish between different habitat types.

### From Raw Data to Usable Information

Satellite data doesn't come ready to use. The raw measurements include contributions from:

- **The atmosphere** — scattering and absorption by gases, aerosols, and water vapor
- **Sun-sensor geometry** — the angles of illumination and viewing affect apparent brightness
- **The water surface** — sun glint and reflection at the air-water interface
- **The water column** — absorption and scattering by water itself
- **The seafloor** — what we actually want to measure!

The Tanager data we downloaded is **top-of-atmosphere (TOA) radiance** — the total light that reached the sensor. To extract meaningful information about the seafloor, we need **atmospheric correction** to remove the atmospheric (and for water, surface and water column) contributions.

The result is **remote sensing reflectance (Rrs)** — a physically meaningful quantity representing light leaving the water. This is what Acolite will compute for us.

For water applications, atmospheric correction is especially critical. When a satellite looks at the ocean, **85-95% of the light it detects never even touched the water** — it's sunlight scattered back by the atmosphere. The actual signal from the water (and seafloor) is just 5-15% of the total. Without correction, we'd be trying to see through overwhelming atmospheric "noise."

### What You'll Learn

1. What hyperspectral data is and why it matters for coastal applications
2. How to load and visualize Planet Tanager data using HyperCoast
3. How to extract and interpret spectral signatures from different benthic types
4. Why atmospheric correction is critical for water applications
5. How to classify benthic habitats using spectral data
6. The quantitative advantage of hyperspectral over multispectral data

---

**Study Area:** Abu Dhabi, UAE - Persian Gulf coast  
**Why this location:** Crystal-clear shallow waters, distinct benthic types (sand, seagrass beds, darker substrates), minimal river input means low turbidity

**Estimated time:** 15-20 minutes

---

## 1. Background: Hyperspectral vs. Multispectral Remote Sensing

### What's the Difference?

Imagine you're trying to identify different types of fabric by their color:

- **Multispectral** is like seeing in **RGB** - you can tell red from blue, but two shades of burgundy might look identical
- **Hyperspectral** is like seeing in **400+ colors** - suddenly you can distinguish between cotton burgundy, silk burgundy, and wool burgundy

| Feature | Multispectral (e.g., Sentinel-2) | Hyperspectral (Tanager) |
|---------|----------------------------------|-------------------------|
| Number of bands | ~10 | 426 |
| Band width | Broad (~20-100nm) | Narrow (~5nm) |
| Spectral range | Visible + some NIR/SWIR | 380-2500nm (continuous) |
| Spatial resolution | 10-60m | ~30m |
| Absorption features | Missed or averaged out | Captured precisely |

### Why Does This Matter for Underwater Mapping?

Different materials absorb light at specific wavelengths:
- **Chlorophyll** (in seagrass and algae) absorbs strongly around **675nm**
- **Water** absorbs increasingly in the **red and near-infrared**
- **Sand** reflects broadly across the visible spectrum

With 426 narrow bands, hyperspectral sensors can detect these subtle absorption features that get smoothed out in broad multispectral bands.

### Planet Tanager Mission

Planet's Tanager satellite, launched in **August 2024**, represents a new generation of commercial hyperspectral imaging:

- **426 spectral bands** from 380-2500nm (VSWIR range)
- **~5nm spectral resolution** - narrow enough to capture absorption features
- **~30m ground sample distance** - suitable for habitat-scale mapping
- **Global coverage** - enabling consistent monitoring worldwide

---

## 2. Setup and Data Access

Let's start by importing the libraries we'll need and downloading our study scene.

In [30]:
# Core libraries
import hypercoast
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os
import warnings

# For classification
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Configure proxy for JupyterHub map rendering
# This is needed for interactive maps to display on remote Jupyter servers
os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = 'proxy/{port}'

# Suppress some warnings for cleaner output
warnings.filterwarnings('ignore', category=RuntimeWarning)

print(f"HyperCoast version: {hypercoast.__version__}")

HyperCoast version: 0.18.0


### Download the Tanager Scene

We'll use a scene from Abu Dhabi, UAE captured on May 11, 2025. This scene shows the shallow coastal waters of the Persian Gulf, which are known for their clarity and diverse benthic habitats.

The data is stored as an **HDF5 file** containing top-of-atmosphere (TOA) radiance values.

In [31]:
# Abu Dhabi scene - clear shallow waters with visible benthic features
url = "https://storage.googleapis.com/open-cogs/planet-stac/release1-basic-radiance/20250511_074311_00_4001_basic_radiance.h5"

# Download the file (this may take a minute - the file is ~2GB)
print("Downloading Tanager scene... this may take a few minutes.")
filepath = hypercoast.download_file(url)
print(f"\nDownload complete! File saved to: {filepath}")

20250511_074311_00_4001_basic_radiance.h5 already exists. Skip downloading. Set overwrite=True to overwrite.

Download complete! File saved to: /home/jupyter/edc-tanager-demos/20250511_074311_00_4001_basic_radiance.h5


### Read the Tanager Data

HyperCoast provides a convenient function to read Tanager HDF5 files into an xarray Dataset.

In [32]:
# Read the Tanager data directly from HDF5
# Note: We read directly from the HDF5 file because the STAC metadata format 
# has recently changed and hypercoast.read_tanager() may encounter issues.

import h5py

def read_tanager_direct(filepath):
    """
    Read Tanager HDF5 file directly and construct wavelength array.
    
    Tanager has 426 spectral bands spanning ~380-2500nm (VSWIR range).
    """
    with h5py.File(filepath, 'r') as f:
        # Read the radiance data
        data = f["HDFEOS/SWATHS/HYP/Data Fields/toa_radiance"][()]
        lat = f["HDFEOS/SWATHS/HYP/Geolocation Fields/Latitude"][()]
        lon = f["HDFEOS/SWATHS/HYP/Geolocation Fields/Longitude"][()]
    
    # Construct wavelength array based on Tanager specs
    # 426 bands from ~380nm to ~2500nm
    n_bands = data.shape[0]
    wavelengths = np.linspace(380, 2500, n_bands)
    
    # Create xarray Dataset
    ds = xr.Dataset(
        data_vars={
            "toa_radiance": (("wavelength", "y", "x"), data)
        },
        coords={
            "wavelength": wavelengths,
            "latitude": (("y", "x"), lat),
            "longitude": (("y", "x"), lon),
        },
        attrs={
            "source": "Planet Tanager HDF5",
            "units": "radiance",
        }
    )
    
    return ds

# Read the data
dataset = read_tanager_direct(filepath)

# Let's explore what we have
print("Dataset overview:")
print(dataset)

Dataset overview:
<xarray.Dataset> Size: 651MB
Dimensions:       (wavelength: 426, y: 624, x: 607)
Coordinates:
  * wavelength    (wavelength) float64 3kB 380.0 385.0 ... 2.495e+03 2.5e+03
    latitude      (y, x) float64 3MB 24.19 24.19 24.19 ... 23.99 23.99 23.99
    longitude     (y, x) float64 3MB 53.67 53.67 53.67 ... 53.86 53.86 53.86
Dimensions without coordinates: y, x
Data variables:
    toa_radiance  (wavelength, y, x) float32 645MB 109.2 99.28 ... 0.1329 0.1175
Attributes:
    source:   Planet Tanager HDF5
    units:    radiance


In [33]:
# Check the wavelengths available
if 'wavelength' in dataset.coords:
    wavelengths = dataset.coords['wavelength'].values
    print(f"Number of spectral bands: {len(wavelengths)}")
    print(f"Wavelength range: {wavelengths.min():.1f} - {wavelengths.max():.1f} nm")
    print(f"Average band width: {np.mean(np.diff(wavelengths)):.2f} nm")

Number of spectral bands: 426
Wavelength range: 380.0 - 2500.0 nm
Average band width: 4.99 nm


**Key things to note:**
- The data contains **top-of-atmosphere (TOA) radiance** - the raw energy measured by the sensor
- We have **426 spectral bands** spanning the visible to shortwave infrared
- The data is **not gridded** to a regular coordinate system (HyperCoast handles the interpolation for visualization)
- Later, we'll need to perform **atmospheric correction** to convert this to surface reflectance

---

## 3. Initial Visualization

Let's create an interactive map to explore our scene. We'll select three bands to create a "natural color" composite similar to what our eyes would see.

In [34]:
# Create an interactive map
m = hypercoast.Map()

# Add the Tanager data using wavelengths for RGB display
# Red ~650nm, Green ~550nm, Blue ~450nm
m.add_tanager(
    dataset, 
    wavelengths=[650, 550, 450],  # Specify wavelengths directly for RGB
    vmin=0, 
    vmax=120,  # Adjust these values if the image appears too dark or bright
    layer_name="Tanager RGB"
)

m

Map(center=[24.0911165, 53.7631495], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

**Explore the scene!**

Take a moment to zoom and pan around. Notice:
- The **coastline** separating land from water
- **Shallow areas** where you can see the bottom (lighter blues/greens)
- **Deeper areas** that appear darker blue
- **Different bottom types** - can you spot areas that look different from each other?

### Understanding Band/Wavelength Selection

With 426 bands to choose from, selecting which three to visualize can be overwhelming. You can specify bands by **wavelength** (in nm) or by **band index**. Using wavelengths is more intuitive:

| Color | Wavelength | Description |
|-------|------------|-------------|
| Blue | ~450nm | Visible blue light |
| Green | ~550nm | Visible green light |
| Red | ~650nm | Visible red light |
| NIR | ~850nm | Near-infrared (invisible to eyes) |

Let's try a false-color composite to highlight vegetation:

In [35]:
# False color composite: NIR, Red, Green
# Vegetation appears bright in this combination
m2 = hypercoast.Map()
m2.add_tanager(
    dataset, 
    wavelengths=[850, 650, 550],  # NIR, Red, Green
    vmin=0, 
    vmax=120,
    layer_name="False Color (NIR-R-G)"
)
m2

Map(center=[24.0911165, 53.7631495], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

In this false-color view:
- **Vegetation on land** appears bright red/pink (high NIR reflectance)
- **Water** appears dark (water absorbs NIR strongly)
- Any **aquatic vegetation** in shallow water may show subtle differences

---

## 4. Exploring Spectral Signatures (Before Atmospheric Correction)

Now comes the exciting part! HyperCoast lets us **click on any pixel** and instantly see its full spectral signature - all 426 bands plotted as a curve.

This is the real power of hyperspectral data: instead of just 3 color values, we get a complete spectrum for every pixel.

In [36]:
# Create a map with the spectral signature tool
m3 = hypercoast.Map()
m3.add_tanager(
    dataset, 
    wavelengths=[650, 550, 450],  # RGB display
    vmin=0, 
    vmax=120,
    layer_name="Tanager RGB"
)

# Add the spectral signature widget
# This enables clicking on the map to extract and plot spectra
m3.add("spectral")

m3

Map(center=[24.0911165, 53.7631495], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

### Try These Experiments:

Click on different features and observe how the spectral signatures differ:

1. **Deep water** - What does the spectrum look like? (Should be low values, especially in red/NIR)
2. **Shallow water over sand** - How does it differ from deep water?
3. **Shallow water over darker substrate** - Maybe seagrass or algae?
4. **Land/vegetation** - Very different signature with high NIR reflectance

### What You're Seeing

**Important:** These spectra show **radiance** values - the raw energy measured by the sensor. This includes:
- Light reflected from the surface we want to study
- Light scattered by the atmosphere (haze, aerosols)
- Light absorbed and re-emitted by atmospheric gases

Think of it like looking through a dirty window - we're seeing the scene, but the window is adding its own "signature" to everything we observe.

**To get accurate measurements of the seafloor, we need to "clean the window" through atmospheric correction.**

---

## 5. Atmospheric Correction with Acolite

### Why Is This Critical?

When sunlight travels from space to the seafloor and back to the satellite, it passes through:

1. **Atmosphere (down)** - scattered and absorbed
2. **Water surface** - some reflected, some penetrates
3. **Water column** - absorbed and scattered
4. **Seafloor** - reflected based on bottom type (THIS IS WHAT WE WANT)
5. **Water column (up)** - more absorption and scattering
6. **Atmosphere (up)** - more scattering

**Atmospheric correction removes the atmospheric contribution** so we can focus on what's actually happening at the surface and below.

For water applications, this is even more critical because:
- Water-leaving radiance is typically only **5-15%** of the total signal
- The rest is atmospheric path radiance and surface glint
- Small errors in atmospheric correction = large errors in water analysis

### Acolite: Atmospheric Correction for Aquatic Remote Sensing

[Acolite](https://github.com/acolite/acolite) is a specialized processor developed by RBINS (Royal Belgian Institute of Natural Sciences) for aquatic applications. It uses a "dark spectrum fitting" approach:

1. Finds dark pixels (deep water) where surface reflectance should be ~0 in NIR
2. Uses these to estimate the atmospheric contribution
3. Subtracts the atmosphere to get surface reflectance
4. Can also derive water quality parameters (chlorophyll, suspended matter, etc.)

Let's run Acolite on our Tanager scene:

In [37]:
# Acolite is pre-installed on this system
acolite_dir = "/opt/acolite/acolite_py_linux"

# Set up output directory in our home folder
out_dir = os.path.expanduser("~/acolite_output")
os.makedirs(out_dir, exist_ok=True)

print(f"Acolite location: {acolite_dir}")
print(f"Output directory: {out_dir}")

Acolite location: /opt/acolite/acolite_py_linux
Output directory: /home/jupyter/acolite_output


In [38]:
# Run atmospheric correction
# This will take several minutes - a good time to stretch!

print("Starting atmospheric correction with Acolite...")
print("This typically takes 5-10 minutes for a full Tanager scene.")
print("\n" + "="*60)

hypercoast.run_acolite(
    acolite_dir=acolite_dir,
    input_file=filepath,
    out_dir=out_dir,
    l2w_parameters="Rrs_*",  # Remote sensing reflectance for all bands
    rgb_rhot=True,           # Save RGB of TOA reflectance
    rgb_rhos=True,           # Save RGB of surface reflectance
    map_l2w=True,            # Generate maps of water parameters
)

print("\n" + "="*60)
print("Atmospheric correction complete!")

Starting atmospheric correction with Acolite...
This typically takes 5-10 minutes for a full Tanager scene.

Running ACOLITE processing - Generic Version 20231023.0
Python - linux - 3.11.6 | packaged by conda-forge | (main, Oct  3 2023, 11:02:03) [GCC 12.3.0]
Platform - Linux 5.10.0-37-cloud-amd64 - x86_64 - #1 SMP Debian 5.10.247-1 (2025-12-11)
Run ID - 20260118_034512
Identified /home/jupyter/edc-tanager-demos/20250511_074311_00_4001_basic_radiance.h5 as None type
/home/jupyter/edc-tanager-demos/20250511_074311_00_4001_basic_radiance.h5 not recognized.
Not deleting extracted file as "delete_extracted_input" is not in settings.
Not removing text files as "delete_acolite_run_text_files" is not in settings.
Not testing output directory as "delete_acolite_output_directory" is not in settings.

Atmospheric correction complete!


In [39]:
# Let's see what Acolite produced
print("Acolite output files:")
for f in sorted(os.listdir(out_dir)):
    print(f"  {f}")

Acolite output files:
  acolite_run_20260118_032226_l1r_settings_user.txt
  acolite_run_20260118_032226_log_file.txt
  acolite_run_20260118_032226_settings_user.txt
  acolite_run_20260118_034512_l1r_settings_user.txt
  acolite_run_20260118_034512_log_file.txt
  acolite_run_20260118_034512_settings_user.txt


### Understanding Acolite Outputs

Acolite produces several types of files:

- **`*_L1R.nc`** - Level 1R: TOA radiance in NetCDF format
- **`*_L2R.nc`** - Level 2R: Surface reflectance (atmosphere removed)
- **`*_L2W.nc`** - Level 2W: Water-leaving reflectance (Rrs) and derived products
- **`.png` files** - Quick-look RGB images at different processing levels

For benthic mapping, we want the **`*_L2W.nc`** file, which contains:
- **Rrs** (Remote sensing reflectance) - the fraction of downwelling light that is reflected upward from just below the water surface
- This is the standard product for ocean color science

In [40]:
# Find and load the L2W file
l2w_files = [f for f in os.listdir(out_dir) if f.endswith('_L2W.nc')]

if l2w_files:
    l2w_path = os.path.join(out_dir, l2w_files[0])
    print(f"Loading: {l2w_path}")
    
    # Load the corrected data
    ds_corrected = xr.open_dataset(l2w_path)
    print("\nCorrected dataset variables:")
    print([v for v in ds_corrected.data_vars if v.startswith('Rrs')][:10], "... and more")
else:
    print("L2W file not found - Acolite may still be processing or encountered an issue.")
    print(f"Check the output directory: {out_dir}")

L2W file not found - Acolite may still be processing or encountered an issue.
Check the output directory: /home/jupyter/acolite_output


---

## 6. Comparing Before and After Atmospheric Correction

Let's visualize the difference between TOA radiance (what the sensor saw) and surface reflectance (what the surface actually looks like).

In [41]:
# Display the RGB images side by side
from IPython.display import Image, display
from PIL import Image as PILImage

# Find RGB images
rgb_files = [f for f in os.listdir(out_dir) if 'RGB' in f and f.endswith('.png')]
print("Available RGB images:")
for f in rgb_files:
    print(f"  {f}")

Available RGB images:


In [42]:
# Display comparison if images exist
if rgb_files:
    fig, axes = plt.subplots(1, min(len(rgb_files), 2), figsize=(14, 7))
    
    if len(rgb_files) == 1:
        axes = [axes]
    
    for ax, fname in zip(axes, rgb_files[:2]):
        img = plt.imread(os.path.join(out_dir, fname))
        ax.imshow(img)
        # Clean up the filename for title
        title = fname.replace('_', ' ').replace('.png', '')
        ax.set_title(title, fontsize=10)
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("No RGB images found in output directory.")

No RGB images found in output directory.


The atmospheric correction should make the image appear:
- **Clearer** - reduced haze effect
- **More contrast** - better separation between features
- **Darker water** - atmospheric path radiance removed

Now the spectral signatures we extract will represent the actual surface reflectance rather than what the sensor measured through the atmosphere.

---

## 7. Spectral Signatures of Benthic Habitats

Now that we have atmospherically corrected data, let's extract and compare spectral signatures from different benthic types.

First, let's prepare the data for analysis.

In [43]:
# Extract Rrs bands from the corrected dataset
if 'ds_corrected' in dir():
    # Get all Rrs variable names and their wavelengths
    rrs_vars = [v for v in ds_corrected.data_vars if v.startswith('Rrs_')]
    
    # Extract wavelength from variable name (e.g., 'Rrs_443' -> 443)
    wavelengths_rrs = []
    for v in rrs_vars:
        try:
            wl = float(v.split('_')[1])
            wavelengths_rrs.append(wl)
        except:
            pass
    
    wavelengths_rrs = sorted(wavelengths_rrs)
    print(f"Number of Rrs bands: {len(wavelengths_rrs)}")
    print(f"Wavelength range: {min(wavelengths_rrs):.0f} - {max(wavelengths_rrs):.0f} nm")
else:
    print("Corrected dataset not loaded. Please check Acolite output.")

Corrected dataset not loaded. Please check Acolite output.


In [44]:
# Stack Rrs bands into a 3D array (y, x, wavelength)
if 'ds_corrected' in dir() and len(wavelengths_rrs) > 0:
    # Sort wavelengths and corresponding variable names
    sorted_vars = [f'Rrs_{int(wl)}' for wl in sorted(wavelengths_rrs)]
    sorted_vars = [v for v in sorted_vars if v in ds_corrected.data_vars]
    
    # Stack into a single array
    rrs_stack = np.stack([ds_corrected[v].values for v in sorted_vars], axis=-1)
    
    print(f"Rrs data shape: {rrs_stack.shape} (rows, cols, bands)")
    
    # Get coordinates
    lats = ds_corrected['lat'].values if 'lat' in ds_corrected.coords else None
    lons = ds_corrected['lon'].values if 'lon' in ds_corrected.coords else None

### Extracting Sample Spectra

Let's sample a few locations to see the spectral differences between benthic types.

In [45]:
def extract_spectrum(data, row, col, window=3):
    """
    Extract average spectrum from a small window around a pixel.
    Using a window reduces noise from individual pixels.
    """
    half_w = window // 2
    r_start, r_end = max(0, row-half_w), min(data.shape[0], row+half_w+1)
    c_start, c_end = max(0, col-half_w), min(data.shape[1], col+half_w+1)
    
    # Extract window and compute mean, ignoring NaN values
    window_data = data[r_start:r_end, c_start:c_end, :]
    spectrum = np.nanmean(window_data, axis=(0, 1))
    
    return spectrum

# Define sample locations (you may need to adjust these based on the scene)
sample_locations = {
    'Deep Water': (400, 200),
    'Shallow Sand': (300, 400),
    'Dark Substrate': (350, 350),
    'Very Shallow': (250, 500)
}

print("Sample locations defined. Extracting spectra...")

Sample locations defined. Extracting spectra...


In [46]:
# Extract and plot spectra
if 'rrs_stack' in dir():
    fig, ax = plt.subplots(figsize=(12, 6))
    
    colors = plt.cm.tab10.colors
    wavelengths_sorted = sorted(wavelengths_rrs)
    
    for i, (name, (row, col)) in enumerate(sample_locations.items()):
        row = min(row, rrs_stack.shape[0] - 1)
        col = min(col, rrs_stack.shape[1] - 1)
        
        spectrum = extract_spectrum(rrs_stack, row, col)
        
        if not np.all(np.isnan(spectrum)):
            ax.plot(wavelengths_sorted, spectrum, 
                    label=name, color=colors[i], linewidth=2)
    
    ax.set_xlabel('Wavelength (nm)', fontsize=12)
    ax.set_ylabel('Remote Sensing Reflectance (Rrs)', fontsize=12)
    ax.set_title('Spectral Signatures of Different Benthic Types', fontsize=14)
    ax.legend(loc='upper right', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    # Add key wavelength markers
    key_wavelengths = {443: 'Blue', 550: 'Green', 675: 'Chl-a absorption', 700: 'Red edge'}
    for wl, label in key_wavelengths.items():
        if min(wavelengths_sorted) <= wl <= max(wavelengths_sorted):
            ax.axvline(x=wl, color='gray', linestyle='--', alpha=0.5)
            ax.text(wl+5, ax.get_ylim()[1]*0.9, label, fontsize=8, rotation=90)
    
    plt.tight_layout()
    plt.show()
else:
    print("Rrs data not available. Please ensure Acolite processing completed.")

Rrs data not available. Please ensure Acolite processing completed.


### Interpreting the Spectra

| Bottom Type | Spectral Characteristics |
|-------------|-------------------------|
| **Sand** | High overall reflectance, relatively flat in visible, peak in green |
| **Seagrass** | Lower reflectance, absorption feature near 675nm (chlorophyll), possible red edge |
| **Deep Water** | Very low reflectance (signal attenuated by water depth), blue-dominated |
| **Algae-covered** | Variable, often shows chlorophyll absorption, may be brownish |

---

## 8. Benthic Habitat Classification

Let's use **K-means clustering** to classify different benthic habitats based on their spectral signatures.

In [47]:
# Prepare data for classification
if 'rrs_stack' in dir():
    n_rows, n_cols, n_bands = rrs_stack.shape
    rrs_2d = rrs_stack.reshape(-1, n_bands)
    
    print(f"Original shape: {rrs_stack.shape}")
    print(f"Reshaped for clustering: {rrs_2d.shape}")
    
    # Create mask for valid pixels
    valid_mask = np.sum(~np.isnan(rrs_2d), axis=1) > (n_bands * 0.8)
    print(f"Valid pixels: {valid_mask.sum():,} out of {len(valid_mask):,}")

In [48]:
# Perform K-means clustering
if 'valid_mask' in dir() and valid_mask.sum() > 0:
    rrs_valid = rrs_2d[valid_mask].copy()
    
    # Fill NaNs with band means
    for i in range(rrs_valid.shape[1]):
        band_mean = np.nanmean(rrs_valid[:, i])
        rrs_valid[np.isnan(rrs_valid[:, i]), i] = band_mean
    
    # Standardize the data
    scaler = StandardScaler()
    rrs_scaled = scaler.fit_transform(rrs_valid)
    
    # Run K-means with 4 clusters
    n_clusters = 4
    print(f"Running K-means clustering with {n_clusters} clusters...")
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    labels_valid = kmeans.fit_predict(rrs_scaled)
    
    print(f"Clustering complete!")
    print(f"\nCluster sizes:")
    for i in range(n_clusters):
        count = np.sum(labels_valid == i)
        pct = 100 * count / len(labels_valid)
        print(f"  Cluster {i}: {count:,} pixels ({pct:.1f}%)")

In [49]:
# Map labels back to image dimensions
if 'labels_valid' in dir():
    labels_image = np.full(n_rows * n_cols, np.nan)
    labels_image[valid_mask] = labels_valid
    labels_image = labels_image.reshape(n_rows, n_cols)
    print(f"Classification map shape: {labels_image.shape}")

In [50]:
# Visualize the classification
if 'labels_image' in dir():
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    
    wavelengths_sorted = sorted(wavelengths_rrs)
    
    def find_closest_idx(target_wl):
        return np.argmin(np.abs(np.array(wavelengths_sorted) - target_wl))
    
    r_idx, g_idx, b_idx = find_closest_idx(650), find_closest_idx(550), find_closest_idx(480)
    
    # Create RGB composite
    rgb = np.stack([rrs_stack[:, :, r_idx], rrs_stack[:, :, g_idx], rrs_stack[:, :, b_idx]], axis=-1)
    rgb_norm = rgb.copy()
    for i in range(3):
        band = rgb_norm[:, :, i]
        valid = ~np.isnan(band)
        if valid.any():
            p2, p98 = np.nanpercentile(band[valid], [2, 98])
            band = np.clip((band - p2) / (p98 - p2), 0, 1)
            rgb_norm[:, :, i] = band
    rgb_norm = np.nan_to_num(rgb_norm, nan=0)
    
    axes[0].imshow(rgb_norm)
    axes[0].set_title('Corrected RGB Composite (Rrs)', fontsize=14)
    axes[0].axis('off')
    
    cmap = plt.cm.get_cmap('tab10', n_clusters)
    im = axes[1].imshow(labels_image, cmap=cmap, vmin=-0.5, vmax=n_clusters-0.5)
    axes[1].set_title('Hyperspectral Classification (K-means)', fontsize=14)
    axes[1].axis('off')
    cbar = plt.colorbar(im, ax=axes[1], ticks=range(n_clusters), shrink=0.8)
    cbar.ax.set_yticklabels([f'Class {i}' for i in range(n_clusters)])
    
    plt.tight_layout()
    plt.show()

In [51]:
# Plot mean spectrum for each cluster
if 'kmeans' in dir() and 'scaler' in dir():
    centers_scaled = kmeans.cluster_centers_
    centers = scaler.inverse_transform(centers_scaled)
    
    fig, ax = plt.subplots(figsize=(12, 6))
    colors = plt.cm.tab10.colors
    
    for i in range(n_clusters):
        ax.plot(wavelengths_sorted, centers[i], label=f'Cluster {i}', color=colors[i], linewidth=2)
    
    ax.set_xlabel('Wavelength (nm)', fontsize=12)
    ax.set_ylabel('Remote Sensing Reflectance (Rrs)', fontsize=12)
    ax.set_title('Mean Spectral Signature of Each Cluster', fontsize=14)
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)
    ax.axvline(x=675, color='gray', linestyle='--', alpha=0.5)
    ax.text(680, ax.get_ylim()[1]*0.9, 'Chl-a\nabsorption', fontsize=8)
    
    plt.tight_layout()
    plt.show()
    
    print("\nCluster Interpretation Guide:")
    print("- Highest reflectance = likely sand or bright substrate")
    print("- Lowest reflectance = likely deep water")
    print("- Absorption at 675nm = likely vegetation (seagrass/algae)")

---

## 9. The Hyperspectral Advantage: Comparison with Simulated Multispectral

Now comes the key question: **Does having 426 bands actually help?**

In [52]:
# Sentinel-2 band center wavelengths
s2_wavelengths = {'B1': 443, 'B2': 490, 'B3': 560, 'B4': 665, 'B5': 705, 
                  'B6': 740, 'B7': 783, 'B8': 842, 'B8A': 865}

if 'wavelengths_rrs' in dir():
    wavelengths_sorted = sorted(wavelengths_rrs)
    s2_indices = []
    
    print("Mapping Sentinel-2 bands to Tanager bands:")
    for band_name, wl in s2_wavelengths.items():
        idx = np.argmin(np.abs(np.array(wavelengths_sorted) - wl))
        actual_wl = wavelengths_sorted[idx]
        s2_indices.append(idx)
        print(f"  {band_name} ({wl}nm) -> Index {idx} ({actual_wl:.0f}nm)")
    
    print(f"\nSimulated multispectral: {len(s2_indices)} bands")
    print(f"Original hyperspectral: {len(wavelengths_sorted)} bands")

In [53]:
# Create and classify multispectral subset
if 'rrs_stack' in dir() and 's2_indices' in dir():
    rrs_multispectral = rrs_stack[:, :, s2_indices]
    rrs_multi_2d = rrs_multispectral.reshape(-1, len(s2_indices))
    rrs_multi_valid = rrs_multi_2d[valid_mask].copy()
    
    for i in range(rrs_multi_valid.shape[1]):
        band_mean = np.nanmean(rrs_multi_valid[:, i])
        rrs_multi_valid[np.isnan(rrs_multi_valid[:, i]), i] = band_mean
    
    scaler_multi = StandardScaler()
    rrs_multi_scaled = scaler_multi.fit_transform(rrs_multi_valid)
    
    print(f"Running K-means on multispectral data ({len(s2_indices)} bands)...")
    kmeans_multi = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    labels_multi_valid = kmeans_multi.fit_predict(rrs_multi_scaled)
    
    labels_multi_image = np.full(n_rows * n_cols, np.nan)
    labels_multi_image[valid_mask] = labels_multi_valid
    labels_multi_image = labels_multi_image.reshape(n_rows, n_cols)
    print("Multispectral classification complete!")

In [54]:
# Compare classifications
if 'labels_image' in dir() and 'labels_multi_image' in dir():
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    axes[0].imshow(rgb_norm)
    axes[0].set_title('RGB Reference', fontsize=14)
    axes[0].axis('off')
    
    cmap = plt.cm.get_cmap('tab10', n_clusters)
    axes[1].imshow(labels_image, cmap=cmap, vmin=-0.5, vmax=n_clusters-0.5)
    axes[1].set_title(f'Hyperspectral Classification\n({len(wavelengths_sorted)} bands)', fontsize=14)
    axes[1].axis('off')
    
    axes[2].imshow(labels_multi_image, cmap=cmap, vmin=-0.5, vmax=n_clusters-0.5)
    axes[2].set_title(f'Multispectral Classification\n({len(s2_indices)} bands)', fontsize=14)
    axes[2].axis('off')
    
    plt.suptitle('Hyperspectral vs Multispectral Benthic Classification', fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()

In [55]:
# Quantify difference
if 'labels_valid' in dir() and 'labels_multi_valid' in dir():
    from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
    
    ari = adjusted_rand_score(labels_valid, labels_multi_valid)
    nmi = normalized_mutual_info_score(labels_valid, labels_multi_valid)
    
    print("Classification Similarity Metrics:")
    print("="*50)
    print(f"Adjusted Rand Index (ARI): {ari:.3f}")
    print(f"Normalized Mutual Information (NMI): {nmi:.3f}")
    print(f"\nThe classifications agree {ari*100:.1f}% more than random chance.")
    
    if ari < 0.7:
        print("\nThis substantial difference suggests hyperspectral data")
        print("captures information that multispectral sensors miss!")

---

## 10. Summary and Conclusions

### What We Accomplished

1. **Loaded and visualized** Planet Tanager hyperspectral imagery using HyperCoast
2. **Explored spectral signatures** of different features interactively
3. **Performed atmospheric correction** using Acolite
4. **Extracted and interpreted** spectral signatures of benthic habitat types
5. **Classified benthic habitats** using K-means clustering
6. **Compared results** between hyperspectral (426 bands) and simulated multispectral (9 bands)

### Key Takeaways

**Hyperspectral data provides:**
- Detailed spectral information for finer discrimination
- Detection of subtle absorption features (like chlorophyll at 675nm)
- Better separation of spectrally similar materials

**Atmospheric correction is essential** - the signal from water/seafloor is only 5-15% of what the sensor sees.

---

## Data & Software Attribution

**Tanager Data**: Tanager STAC Data, available at www.planet.com/data/stac - 2025 Planet Labs PBC. All Rights Reserved.

**HyperCoast**: Liu, B., & Wu, Q. (2024). HyperCoast: A Python Package for Visualizing and Analyzing Hyperspectral Data in Coastal Environments. Journal of Open Source Software, 9(100), 7025. https://doi.org/10.21105/joss.07025

**Acolite**: Vanhellemont, Q., & Ruddick, K. (2018). Atmospheric correction of metre-scale optical satellite data for inland and coastal water applications. Remote Sensing of Environment, 216, 586-597.

---
